In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from time import time
from sklearn.tree import DecisionTreeRegressor
import gc
import joblib

# Разработка модели машинного обучения

Загрузим данные

In [2]:
X = pd.read_feather('/home/c4/Рабочий стол/C4_M4/data_X')
y = pd.read_feather('/home/c4/Рабочий стол/C4_M4/data_y')

Разделим наши данные на обучающую и тестовую выборку

In [3]:
RANDOM_SEED = 5022025

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=RANDOM_SEED)

Закон больших чисел утверждает, что при увеличении объема выборки результаты становятся более предсказуемыми и менее подвержены случайным колебаниям. Так как у нас 120 миллионов строк тренировочных данных, то обучим наши модели только на части данных.

In [5]:
X_train_sh = X_train.sample(frac=0.4, random_state=RANDOM_SEED)

In [6]:
idx = X_train_sh.index.to_list()

In [7]:
y_train_sh = y_train.loc[y_train.index.isin(idx)]

## Линейная модель

Ввиду простоты линейной модели, то подадим ей на вход все тренировочные данные

In [8]:
lin_model = LinearRegression(n_jobs=-1)

In [9]:
start = time()
lin_model.fit(X_train,y_train)
end = time()

In [10]:
y_pred = lin_model.predict(X_test)

In [11]:
r2_lin = r2_score(y_test, y_pred)

In [12]:
rmse_lin = root_mean_squared_error(y_test, y_pred)

In [13]:
print(f'Время потраченное на обучение составило {end - start} секунд')

Время потраченное на обучение составило 30.765994787216187 секунд


In [14]:
print(f'Полученные метрики линейной регрессии: R2= {r2_lin}, RMSE= {rmse_lin}')

Полученные метрики линейной регрессии: R2= 0.43254926800727844, RMSE= 930.2916259765625


На тестовой выборке линейная регрессия обученная на тренировочных данных показала $R2 = 0.432$, $RMSE = 930$

Обучим случайный лес

Так как Random Forest не так сильно оптимизирован как алгоритмы градиентного бустинга, и не такая простоая как линейная модель, то обучим ее на малых данных

In [15]:
X_train_sh_1 = X_train.sample(frac=0.1, random_state=RANDOM_SEED)
idx = X_train_sh_1.index.to_list()
y_train_sh_1 = y_train.loc[y_train.index.isin(idx)]

In [26]:
rf = RandomForestRegressor(n_jobs=-1, random_state=RANDOM_SEED, n_estimators=1, bootstrap=True, max_depth=1)

In [27]:
start_rf = time()
rf.fit(X_train_sh_1,y_train_sh_1)
end_rf = time()

In [28]:
y_pred = rf.predict(X_test)

In [29]:
r2_rf = r2_score(y_test, y_pred)
rmse_rf = root_mean_squared_error(y_test,y_pred)

In [30]:
print(f'Время потраченное на обучение составило {end_rf - start_rf} секунд')

Время потраченное на обучение составило 24.489259243011475 секунд


In [31]:
print(f'Полученные метрики линейной регрессии: R2= {r2_rf}, RMSE= {rmse_rf}')

Полученные метрики линейной регрессии: R2= 9.163704340087175e-05, RMSE= 1234.3690066002296


## Градиентные бустинги

In [42]:
lgb = LGBMRegressor(random_state=RANDOM_SEED, n_jobs=-1, n_estimators=100, early_stopping_rounds=5)

In [43]:
start_1 = time()
lgb.fit(X_train_sh, y_train_sh['0'],
       eval_set=(X_test, y_test['0']), eval_metric=['rmse', 'r2'])
end_1 = time()

[LightGBM] [Warning] early_stopping_round is set=5, early_stopping_rounds=5 will be ignored. Current value: early_stopping_round=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.272692 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 38400000, number of used features: 7
[LightGBM] [Warning] early_stopping_round is set=5, early_stopping_rounds=5 will be ignored. Current value: early_stopping_round=5
[LightGBM] [Info] Start training from score 2761.604222
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[4]	valid_0's rmse: 1181.18	valid_0's l2: 1.39519e+06


In [45]:
y_pred_sh = lgb.predict(X_test)

In [46]:
r2_lgb_1 = r2_score(y_test['0'], y_pred_sh)

In [47]:
rmse_lgb_1 = root_mean_squared_error(y_test['0'], y_pred_sh)

In [48]:
start_2 = time()
lgb.fit(X_train_sh, y_train_sh['1'],
       eval_set=(X_test, y_test['1']), eval_metric=['rmse', 'r2'])
end_2 = time()

[LightGBM] [Warning] early_stopping_round is set=5, early_stopping_rounds=5 will be ignored. Current value: early_stopping_round=5
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.088550 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 38400000, number of used features: 7
[LightGBM] [Warning] early_stopping_round is set=5, early_stopping_rounds=5 will be ignored. Current value: early_stopping_round=5
[LightGBM] [Info] Start training from score 3088.561501
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[12]	valid_0's rmse: 1287.59	valid_0's l2: 1.65788e+06


In [49]:
y_pred_sh = lgb.predict(X_test)

In [50]:
r2_lgb_2 = r2_score(y_test['1'], y_pred_sh)

In [51]:
rmse_lgb_2 = root_mean_squared_error(y_test['1'], y_pred_sh)

In [52]:
print(f'Время потраченное на обучение составило {end_2 - start_2 + end_1 - start_1} секунд')

Время потраченное на обучение составило 53.742021322250366 секунд


In [54]:
print(f'Среднее значение метрик для LGBM составило RMSE = {(rmse_lgb_1 + rmse_lgb_2)/2}, R2= {(r2_lgb_1+r2_lgb_2)/2}')

Среднее значение метрик для LGBM составило RMSE = 1234.384614393801, R2= 6.614457621684222e-05


Напишем наш пайплайн

Так как метрики получились гораздо лучше, чем у LGBM и Случайного леса, выберем ее за основную

In [36]:
def pipeline(train_data_path, train_target_path, val_data_path, val_target_path):
    X_train = pd.read_feather(train_data_path)
    X_val = pd.read_feather(val_data_path)
    y_train = pd.read_feather(train_target_path)
    y_val = pd.read_feather(val_target_path)

    model = LinearRegression(n_jobs=-1)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    r2 = r2_score(y_val, y_pred)
    rmse = root_mean_squared_error(y_val, y_pred)
    return [model, {'R2': r2, 'RMSE': rmse}]

In [37]:
model, metric = pipeline(
    train_data_path= '/home/c4/Рабочий стол/C4_M4/data_X',
    train_target_path = '/home/c4/Рабочий стол/C4_M4/data_y',
    val_data_path = '/home/c4/Рабочий стол/C4_M4/data_X_val',
    val_target_path= '/home/c4/Рабочий стол/C4_M4/data_y_val'
)

In [38]:
model

LinearRegression(n_jobs=-1)

In [39]:
metric

{'R2': 0.44386065006256104, 'RMSE': 921.104248046875}

На валидационных данных получили метрику $R2 = 0.444$ и $RMSE = 921$, используя линейную модель 

In [41]:
joblib.dump(model, 'model.pkl')

['model.pkl']

# Вывод

Основная задача состояла в том, чтобы обучить наши тренировочные данные, подготовленные в прошлом модуле и получить метрики на валидационной выборке. Рассматривались три модели: Линейная регрессия, LGBMRegressor, RandomForestRegressor.

Модели были сравнены по точности и скорости:
1. Линейная регрессия: скорость - 31 секунда $R2 = 0.432$, $RMSE= 930$
2. RandomForest Regressor: скорость - 24.5 cсекунды $R2 ≈ 0$, $RMSE= 1234$
3. LGBM: скорость - 54 секунды $R2 ≈ 0$, $RMSE= 1235$

Линейная модель отработала медленее, чем Случайный лес, но гораздо точнее. Поэтому она была выбрана в качестве основной модели.
В конце был написан пайплайн по обучению модели и предсказанию ее на валидационных данных. 
На валидационных данных получили метрику $R2 = 0.444$ и $RMSE = 921$